In [1]:
import os
import pandas as pd

In [2]:
# Execute this block only once.
# sys.path.append(os.path.dirname(os.getcwd()))
os.chdir("..")

In [3]:
evaluation_dataframe = pd.read_csv("tests/dataset/scored_data.csv")

In [4]:
evaluation_dataframe

,Unnamed: 0,prompt_type,prompt,expected_response,received_response,answer_type,score,answerable
0,0,Stock Performance,What was the closing price of ORCL on 2014-05-19?,$42.16,The closing price of ORCL on 2014-05-19 was $4...,Text,10,1
1,1,Stock Performance,What was the closing price of DPS on 2014-06-24?,$58.56,The closing price of DPS on 2014-06-24 was $58...,Text,10,1
2,2,Stock Performance,What was the closing price of ARNC on 2017-12-13?,$25.18,The closing price of ARNC on 2017-12-13 was $2...,Text,10,1
3,3,Stock Performance,What was the closing price of NBL on 2015-11-25?,$36.94,"The closing price of NBL on November 25, 2015 ...",Text,10,1
4,4,Stock Performance,What was the closing price of ILMN on 2015-07-06?,$220.51,The closing price of ILMN on 2015-07-06 was $2...,Text,10,1
...,...,...,...,...,...,...,...,...
132,132,Visualization,Generate a Plotly box plot showing the distrib...,showing the box plot for the distribution of c...,showing the box plot for the distribution of c...,Code,8,1
133,133,Visualization,Generate a Plotly bar chart showing the total ...,showing the bar chart for the total trading vo...,showing the Plotly bar chart displaying the to...,Code,10,1
134,134,Visualization,Plot a Plotly histogram showing the distributi...,showing the histogram for the distribution of ...,showing the histogram for the distribution of ...,Code,9,1
135,135,Visualization,Create a Plotly scatter plot comparing the clo...,The scatter plot comparing the closing price a...,The scatter plot comparing the closing price a...,Code,8,1


In [5]:
print("Evaluation Score: ", format(
            evaluation_dataframe['score'].sum() / ( len(evaluation_dataframe) * 10) * 100, '0.2f'
        ) + "%"
    )

Evaluation Score:  63.43%


In [6]:
from io import BytesIO
import streamlit as st
from agentlite.logging.streamlit_logger import UILogger
from agentlite.llm.LLMConfig import LLMConfig
from agentlite.llm.agent_llms import get_llm_backend
from agentlite_finance.memory.shared_memory import SharedMemory
from agentlite_finance.actions.code_generation_action import CodeGenerationAction
from agentlite_finance.actions.file_handler_action import FileHandlerAction
from agentlite_finance.actions.generic_insights_action import GenericInsightsAction
from agentlite_finance.actions.plotting_action import PlottingAction
from agentlite.logging.terminal_logger import AgentLogger
from agentlite_finance.examples.agents_example import AgentsExample
from agentlite_finance.examples.manager_example import ManagerExample
from agentlite_finance.actions.preprocessing_action import PreprocessingAction
from agentlite.commons import TaskPackage
from agentlite_finance.memory.memory_keys import FILE

llm_config_dict = {
                    "llm_name": "gpt-3.5-turbo",
                    "temperature": 0.0,
                    "max_tokens": 2000,
                    "api_key": os.getenv('OPENAI_API_KEY')
                    }
llm_config = LLMConfig(llm_config_dict)
llm = get_llm_backend(llm_config)

# define the logger. You can set the PROMPT_DEBUG_FLAG=True to show the actual prompt and generation for LLM
agent_logger = AgentLogger(PROMPT_DEBUG_FLAG=False, FLAG_PRINT=False)
agent_logger = UILogger()

stock_file_path = "tests/dataset/all_stocks_5yr.csv"
stock_data_file = open(stock_file_path, "rb")

stock_data_bytesio = BytesIO(stock_data_file.read())
stock_data_bytesio.name = stock_file_path.split('/')[-1]

# Initialize agents
shared_mem = SharedMemory()
shared_mem.add(FILE, stock_data_bytesio)

file_handler_action = FileHandlerAction(shared_mem)
preprocessing_action = PreprocessingAction(shared_mem)

from agentlite_finance.agents.data_agent import DataAgent
data_agent = DataAgent(
    llm=llm,
    actions=[file_handler_action, preprocessing_action],
    shared_mem=shared_mem,
    logger=agent_logger
    )

codegeneration_action = CodeGenerationAction(shared_mem)
plotting_action = PlottingAction(shared_mem)
from agentlite_finance.agents.visualization_agent import VisualizationAgent
visualization_agent = VisualizationAgent(
    llm=llm,
    actions=[codegeneration_action, plotting_action],
    shared_mem=shared_mem,
    logger=agent_logger
    )

generic_insights_action = GenericInsightsAction(shared_mem)
from agentlite_finance.agents.generic_agent import GenericAgent
generic_agent = GenericAgent(
    llm=llm,
    actions=[generic_insights_action],
    shared_mem=shared_mem,
    logger=agent_logger
    ) 

example_task1, act_chain1 = AgentsExample().build_data_agent_example()
data_agent.add_example(task=example_task1, action_chain=act_chain1)
example_task2, act_chain2 = AgentsExample().build_visualisation_example()
visualization_agent.add_example(task=example_task2, action_chain=act_chain2)
example_task3, act_chain3 = AgentsExample().build_summary_example()
generic_agent.add_example(task=example_task3, action_chain=act_chain3)

if "messages" not in st.session_state:
    st.session_state.messages = []

# Manager agent
from agentlite_finance.manager.finance_data_manager import FinanceDataManagerAgent
finance_data_manager = FinanceDataManagerAgent(
    llm=llm,
    team=[
            data_agent ,visualization_agent, generic_agent                    
        ],
    logger=agent_logger
)
manager_task, manager_chain = ManagerExample().build_manager_example()
finance_data_manager.add_example(task=manager_task, action_chain=manager_chain)

summary_task, summary_chain = ManagerExample().build_manager_example_for_data_summary()
finance_data_manager.add_example(task=summary_task, action_chain=summary_chain)

2024-10-12 12:10:05.999 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 12:10:06.000 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-10-12 12:10:06.001 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 12:10:06.001 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [7]:
import re
import dspy
from pydantic import BaseModel, Field

class Score(BaseModel):
    commentary: str = Field(desc="Analysis related to the visualization code score")
    Score: int = Field(desc="Score of the visualization code")

class VisualizationScorer(dspy.Signature):
    """
    You are a visualization code evaluating agent. You take a prompt and 
    visualization code for evaluation. 
    You need to +1 for each of these attributes in the visualization code

    {'correct_column_names', 'title', 'Annotations', 'Aggregation used',
    'correct axis label', 'Plotly_white theme', 'Correct chart type'}

    You are provided with a {prompt}
    and Plotly visualization code {code}
    You need to provide the total score
    """
    prompt = dspy.InputField(desc="user prompt which asks specific information about data and required chart")
    code = dspy.InputField(desc="The agent generated visualization code")
    output: Score = dspy.OutputField(desc='The total score after evaluating the visualization code')

class AnswerScorer(dspy.Signature):
    """
    You are a text based response evaluating agent. You take a prompt, received response and 
    expected response for evaluation. 
    You need to provide 10 score when the received response matches with the expected response.
    Else provide 0 as score. Do not any other numerical value in the response apart from score.
    Ignore the sentence formation difference and only concentrate on the numerical response
    correctness.

    You are provided with a {prompt},
    expected response {expected_response}
    and received response {received_response}
    You need to provide the total score
    """
    prompt = dspy.InputField(desc="user prompt which asks specific information about data")
    expected_response = dspy.InputField(desc="The response which is expected from agent")
    received_response = dspy.InputField(desc="The agent generated response")
    output: Score = dspy.OutputField(desc='The score after evaluating the received response against expected response')

def extract_integers(text):
    return [int(num) for num in re.findall(r'\d+', text)]

def check_if_python_code_exists(response):
    return "```python" in response

def fetch_python_code( text):
    code = re.findall(r'```python(.*?)```', text, re.DOTALL)
    if code:
        cleaned_code = code[0].strip()
        return cleaned_code
    return None

def python_code_execution_score(response):
    score = 0
    try:
        code = fetch_python_code(response)
        exec(code)
        score += 3
        return score

    except:
        return score

def evaluating_code_response(code, prompt):
    score = 0
    scorer = dspy.Predict(VisualizationScorer)
    response = scorer(prompt=prompt, code=code)
    score+=extract_integers(response.output)[0]
    return score

def evaluating_text_response( prompt, expected_response, received_response):
    score = 0
    scorer = dspy.Predict(AnswerScorer)
    response = scorer(
                        prompt=prompt,
                        expected_response=expected_response,
                        received_response=received_response
                    )
    score+=extract_integers(response.output)[0]
    return score

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/kamalkis/Library/Application Support/sagemaker/config.yaml
2024-10-12 12:10:09 - PyTorch version 2.2.2 available.
PyTorch version 2.2.2 available.


/Users/kamalkis/Documents/Personal/Capstone/poc_cap/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import dspy

class DsPyWrapperOnFinanceAgent(dspy.Module):
    def __init__(self):
        super().__init__()
        self.agent = finance_data_manager
    
    def forward(self, prompt):
        task_pack = TaskPackage(instruction=prompt)
        prediction = self.agent(task_pack)
        return dspy.Prediction(response=prediction)


lm = dspy.OpenAI(
                    model='gpt-3.5-turbo', 
                    api_key=os.getenv('OPENAI_API_KEY')
                )
dspy.configure(lm=lm)
dspy_wrapped_finance_agent=DsPyWrapperOnFinanceAgent()

In [9]:
def total_score(answerable, score):
    return answerable * score

In [10]:
def evaluation_metric(expected, received, trace=None):
    executable_score = python_code_execution_score(received.response)
    attributes_score = evaluating_code_response(received.response, expected.prompt) 
    return executable_score + attributes_score

In [11]:
filtered_dataframe = evaluation_dataframe.query('answer_type == "Code" & score == 10')

In [12]:
trainset = [
                dspy.Example(prompt=prompt, code=code).with_inputs('prompt')
                for prompt, code in zip(
                                            filtered_dataframe['prompt'],
                                            filtered_dataframe['expected_response']
                                        )
            ]

In [13]:
len(trainset)

8

In [14]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

config = dict(max_bootstrapped_demos=2, max_labeled_demos=2, num_candidate_programs=3, num_threads=4, )

teleprompter = BootstrapFewShotWithRandomSearch(metric=evaluation_metric, **config)
optimized_agent = teleprompter.compile(dspy_wrapped_finance_agent, trainset=trainset, )

Going to sample between 1 and 2 traces per predictor.
Will attempt to bootstrap 3 candidate sets.
2024-10-12 12:10:33 - Failed to deep copy attribute 'agent' of DsPyWrapperOnFinanceAgent, falling back to shallow copy or reference copy.
Failed to deep copy attribute 'agent' of DsPyWrapperOnFinanceAgent, falling back to shallow copy or reference copy.


/Users/kamalkis/Documents/Personal/Capstone/poc_cap/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
  0%|          | 0/8 [00:00<?, ?it/s]

2024-10-12 12:10:35 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2024-10-12 12:10:35.080 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'ThreadPoolExecutor-4_2': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 12:10:35.081 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'ThreadPoolExecutor-4_2': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 12:10:35.082 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'ThreadPoolExecutor-4_2': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2024-10-12 12:10:35 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

2024-10-12 12:22:19 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2024-10-12 12:22:19.940 Thread 'ThreadPoolExecutor-9_3': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 12:22:19.943 Thread 'ThreadPoolExecutor-9_3': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 12:22:19.946 Thread 'ThreadPoolExecutor-9_3': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 12:22:19.948 Thread 'ThreadPoolExecutor-9_3': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 12:22:19.953 Thread 'ThreadPoolExecutor-9_3': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 12:22:19.955 Thread 'ThreadPoolExecutor-9_3': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 12:22:19.959 Thread 'ThreadPoolExecutor-9_3': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 12:22:19.970 Thread 'Th

2024-10-12 12:22:20 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Average Metric: 25.0 / 8  (312.5): 100%|██████████| 8/8 [00:48<00:00,  6.05s/it]

2024-10-12 12:22:20 - 2024-10-12T06:52:20.860448Z [info     ] Average Metric: 25.0 / 8 (312.5%) [dspy.evaluate.evaluate] filename=evaluate.py lineno=218
2024-10-12T06:52:20.860448Z [info     ] Average Metric: 25.0 / 8 (312.5%) [dspy.evaluate.evaluate] filename=evaluate.py lineno=218
Scores so far: [800.0, 725.0, 650.0, 25612.5, 700.0, 312.5]
Best score so far: 25612.5
6 candidate programs found.


In [15]:
lm.inspect_history(n=1)




You are a visualization code evaluating agent. You take a prompt and 
visualization code for evaluation. 
You need to +1 for each of these attributes in the visualization code

{'correct_column_names', 'title', 'Annotations', 'Aggregation used',
'correct axis label', 'Plotly_white theme', 'Correct chart type'}

You are provided with a {prompt}
and Plotly visualization code {code}
You need to provide the total score

---

Follow the following format.

Prompt: user prompt which asks specific information about data and required chart
Code: The agent generated visualization code
Output: The total score after evaluating the visualization code

---

Prompt: Generate a Plotly chart showing the Bollinger Bands for all stocks using their closing prices.
Code: Expert insights have been generated from the preprocessed data. Here are some key insights: 1. Price Trends: - The stock prices of Apple show a gradual increase over time, with occasional dips and peaks. - There is a noticeable upward t

"\n\n\nYou are a visualization code evaluating agent. You take a prompt and \nvisualization code for evaluation. \nYou need to +1 for each of these attributes in the visualization code\n\n{'correct_column_names', 'title', 'Annotations', 'Aggregation used',\n'correct axis label', 'Plotly_white theme', 'Correct chart type'}\n\nYou are provided with a {prompt}\nand Plotly visualization code {code}\nYou need to provide the total score\n\n---\n\nFollow the following format.\n\nPrompt: user prompt which asks specific information about data and required chart\nCode: The agent generated visualization code\nOutput: The total score after evaluating the visualization code\n\n---\n\nPrompt: Generate a Plotly chart showing the Bollinger Bands for all stocks using their closing prices.\nCode: Expert insights have been generated from the preprocessed data. Here are some key insights: 1. Price Trends: - The stock prices of Apple show a gradual increase over time, with occasional dips and peaks. - Ther

In [32]:
optimal_response_list = []
for ind, prompt in enumerate(evaluation_dataframe['prompt']):
    while True:
        try:
            optimal_response_list.append( 
                                            optimized_agent(prompt).response
                                        )
            break
        except BaseException:
            print("Retring again.")
evaluation_dataframe['optimal_response'] = optimal_response_list

2024-10-12 13:42:18 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2024-10-12 13:42:18.496 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:42:18.497 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:42:18.498 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:42:18.499 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:42:18.500 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2024-10-12 13:42:19 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2024-10-12 13:42:19.205 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:42:19.206 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:42:19.208 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:42:19.210 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:42:19.211 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:42:19.580 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:42:19.581 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:42:19.581 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2024-10-12 13:42:21 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

2024-10-12 13:45:21.516 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:21.518 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:21.519 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:21.521 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:21.522 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:39.578 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:39.695 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:39.705 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2024-10-12 13:45:45 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2024-10-12 13:45:45.420 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:45.422 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:45.432 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:45.433 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:45.437 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:45.442 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:45.445 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:45.448 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2024-10-12 13:45:52 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2024-10-12 13:45:52.695 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:52.696 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:52.698 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:52.699 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:52.700 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:52.701 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:52.703 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-12 13:45:52.704 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [34]:
evaluation_dataframe['optimal_score'] = [
        evaluating_text_response(expected, received, prompt)
        if answer_type == "Text"
        else evaluating_code_response(received, prompt) + python_code_execution_score(received)
        for expected, received, prompt, answer_type  in zip( 
                evaluation_dataframe['expected_response'],
                evaluation_dataframe['optimal_response'],
                evaluation_dataframe['prompt'],
                evaluation_dataframe['answer_type']
            )
    ]

2024-10-12 14:06:45 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-12 14:06:45 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-12 14:06:46 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-12 14:06:46 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-12 14:06:48 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-10-12 14:06:48 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/

In [35]:
print("Optimal Score after Few Shot Training: ", format(
            evaluation_dataframe['optimal_score'].sum() / ( len(evaluation_dataframe) * 10) * 100, '0.2f'
        ) + "%"
    )

Optimal Score after Few Shot Training:  70.44%


In [36]:
evaluation_dataframe

,Unnamed: 0,prompt_type,prompt,expected_response,received_response,answer_type,score,answerable,optimal_response,optimal_score
0,0,Stock Performance,What was the closing price of ORCL on 2014-05-19?,$42.16,The closing price of ORCL on 2014-05-19 was $4...,Text,10,1,The closing price of ORCL on 2014-05-19 was $4...,10
1,1,Stock Performance,What was the closing price of DPS on 2014-06-24?,$58.56,The closing price of DPS on 2014-06-24 was $58...,Text,10,1,"The closing price of DPS on June 24, 2014 was ...",10
2,2,Stock Performance,What was the closing price of ARNC on 2017-12-13?,$25.18,The closing price of ARNC on 2017-12-13 was $2...,Text,10,1,"The closing price of ARNC on December 13, 2017...",10
3,3,Stock Performance,What was the closing price of NBL on 2015-11-25?,$36.94,"The closing price of NBL on November 25, 2015 ...",Text,10,1,"The closing price of NBL on November 25, 2015 ...",10
4,4,Stock Performance,What was the closing price of ILMN on 2015-07-06?,$220.51,The closing price of ILMN on 2015-07-06 was $2...,Text,10,1,The closing price of ILMN on 2015-07-06 was $2...,10
...,...,...,...,...,...,...,...,...,...,...
132,132,Visualization,Generate a Plotly box plot showing the distrib...,showing the box plot for the distribution of c...,showing the box plot for the distribution of c...,Code,8,1,showing the box plot for the distribution of c...,9
133,133,Visualization,Generate a Plotly bar chart showing the total ...,showing the bar chart for the total trading vo...,showing the Plotly bar chart displaying the to...,Code,10,1,showing the Plotly bar chart for the total tra...,7
134,134,Visualization,Plot a Plotly histogram showing the distributi...,showing the histogram for the distribution of ...,showing the histogram for the distribution of ...,Code,9,1,showing the histogram for the distribution of ...,9
135,135,Visualization,Create a Plotly scatter plot comparing the clo...,The scatter plot comparing the closing price a...,The scatter plot comparing the closing price a...,Code,8,1,showing the scatter plot comparing the closing...,9


In [37]:
evaluation_dataframe.to_csv("tests/dataset/optimised_data.csv")